### Merge Reddit Data

In [ ]:
import pandas as pd
import os
import glob

# Define paths
reddit_raw_path = r"C:\Users\user\Desktop\USAID-Kenya-Sentiment-Analysis\USAID-Kenya-Sentiment-Analysis\data\raw\reddit_data"
output_file = r"C:\Users\user\Desktop\USAID-Kenya-Sentiment-Analysis\USAID-Kenya-Sentiment-Analysis\data\processed\Agatha_merged_reddit_dataset.csv"
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Define final Reddit column structure
final_columns = [
    "post_title", "text", 
    "keyword", "published_date", "url"
]

# Load and normalize all Reddit files
reddit_files = glob.glob(os.path.join(reddit_raw_path, "*.csv"))
reddit_dfs = []

for file in reddit_files:
    df = pd.read_csv(file)

    # ==== Rename columns as needed ====
    if 'title' in df.columns:
        df.rename(columns={'title': 'post_title'}, inplace=True)

    # Handle published_date
    if 'date_posted' in df.columns:
        df['published_date'] = df['date_posted']
    elif 'created_utc' in df.columns:
        df['published_date'] = df['created_utc']

    # Handle text column
    if 'text' not in df.columns and 'selftext' in df.columns:
        df.rename(columns={'selftext': 'text'}, inplace=True)


    # Add missing final columns with None
    for col in final_columns:
        if col not in df.columns:
            df[col] = None

    # Format published_date to datetime
    df['published_date'] = df['published_date'].astype(str).str.strip().replace('', pd.NA)
    df['published_date'] = pd.to_datetime(df['published_date'], errors='coerce')

    # Keep only final standardized columns
    df = df[final_columns]
    reddit_dfs.append(df)

#  Combine all rows
reddit_df = pd.concat(reddit_dfs, ignore_index=True)

# Save to the output location
reddit_df.to_csv(output_file, index=False)

# print the five rows of the dataset
reddit_df.head()


<ipython-input-5-1ac5fb5e64e0>:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['published_date'] = pd.to_datetime(df['published_date'], errors='coerce')
<ipython-input-5-1ac5fb5e64e0>:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['published_date'] = pd.to_datetime(df['published_date'], errors='coerce')
<ipython-input-5-1ac5fb5e64e0>:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['published_date'] = pd.to_datetime(df['published_date'], errors='coerce')


,post_title,text,keyword,published_date,url
0,"USAID left a month ago, do we have ARVs in Kenya?",Someone on a different group (different websit...,usaid kenya,2025-04-15 13:16:53,https://www.reddit.com/r/Kenya/comments/1jzrn2...
1,Classism in r/Kenya and r/nairobi,The classism I'm seeing in both subs is a good...,usaid kenya,2025-04-07 04:21:12,https://www.reddit.com/r/Kenya/comments/1jtcvb...
2,EX-USAID people!! Let's talk,Are you still in contact with the organisation...,usaid kenya,2025-04-05 19:09:10,https://www.reddit.com/r/Kenya/comments/1jsb14...
3,Why western powers back Israel no matter what ...,"I don't care what good book you read, but it's...",usaid kenya,2025-03-25 08:18:04,https://www.reddit.com/r/Kenya/comments/1jjehw...
4,Is kenya capable of funding its needs now that...,How is kenya prepared to fill the vacuum of US...,usaid kenya,2025-03-08 08:08:58,https://www.reddit.com/r/Kenya/comments/1j6cjz...


In [6]:
reddit_df.shape

(1289, 5)

In [7]:
# Shows the number of missing (NaN) values in each column
reddit_df.isna().sum()


post_title          0
text              388
keyword           547
published_date    347
url                 0
dtype: int64

In [8]:
# Find number of duplicate rows
reddit_df.duplicated().sum()

139

### Merge News data

In [ ]:
import pandas as pd
import os
import glob

# Define input and output paths
news_raw_path = r"C:\Users\user\Desktop\USAID-Kenya-Sentiment-Analysis\USAID-Kenya-Sentiment-Analysis\data\raw\news_data"
output_file = r"C:\Users\user\Desktop\USAID-Kenya-Sentiment-Analysis\USAID-Kenya-Sentiment-Analysis\data\processed\Agatha_merged_news_dataset.csv"

os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Final column structure based on guidelines
final_columns = [
    "title", "description", "text", "url",
    "keyword", "published_date"
]

# Process and merge all news data files
news_files = glob.glob(os.path.join(news_raw_path, "*.csv"))
news_dfs = []

for file in news_files:
    df = pd.read_csv(file)

    # Rename/transform relevant columns
    if 'publishedAt' in df.columns:
        df['published_date'] = df['publishedAt']
    if 'content' in df.columns:
        df['text'] = df['content']


    # Fill in missing required columns
    for col in final_columns:
        if col not in df.columns:
            df[col] = None

    # Convert published_date to proper datetime
    df['published_date'] = df['published_date'].astype(str).str.strip().replace('', pd.NA)
    df['published_date'] = pd.to_datetime(df['published_date'], errors='coerce')

    # Retain only final structure
    df = df[final_columns]
    news_dfs.append(df)

# Concatenate all dataframes
news_df = pd.concat(news_dfs, ignore_index=True)

# Save the final merged file
news_df.to_csv(output_file, index=False)

# Print preview
news_df.head()


,title,description,text,url,keyword,published_date
0,Has DOGE really saved the US government $180bn?,Elon Musk first claimed the department would m...,President Donald Trump and adviser Elon Musk c...,https://www.aljazeera.com/news/2025/6/6/has-do...,usaid kenya,2025-06-06 11:21:51+00:00
1,The Life Story of Ecomobilus Technologies Limi...,By Prof Geoffrey Gitau Here is a story showcas...,By Prof Geoffrey Gitau\r\nHere is a story show...,https://cleantechnica.com/2025/05/26/the-life-...,usaid kenya,2025-05-26 17:13:41+00:00
2,"Death, Sexual Violence and Human Trafficking: ...",by Brett Murphy and Anna Maria Barry-Jester \n...,ProPublica is a nonprofit newsroom that invest...,https://www.propublica.org/article/trump-usaid...,usaid kenya,2025-05-28 18:45:00+00:00
3,Congress Should Quickly Approve Trump’s Rescis...,President Donald Trump‘s rescission legislatio...,President Donald Trumps rescission legislation...,https://www.dailysignal.com/2025/06/10/congres...,usaid kenya,2025-06-10 12:00:00+00:00
4,Food Safety Depends On Every Link In The Suppl...,Almost 1 in 10 people globally fall ill from c...,Colorful fish and vegetables can be purchased ...,https://www.forbes.com/sites/daniellenierenber...,usaid kenya,2025-06-06 13:55:41+00:00


In [2]:
news_df.shape

(2549, 6)

In [3]:
# Shows the number of missing (NaN) values in each column
news_df.isna().sum()

title               0
description        16
text               25
url                 2
keyword           170
published_date     99
dtype: int64

In [4]:
# Find number of duplicate rows
news_df.duplicated().sum()

111